In [1]:
import pandas as pd
import numpy as np
import csv as csv
import math
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cross_validation import train_test_split
import time
from datetime import date
import datetime
%matplotlib inline

def saveFileForSubmission(predicted_lables,custonFileName='submission.csv',customHeader=''):
    result = np.c_[predicted_lables]

    np.savetxt(custonFileName, 
           result.astype(int), 
           delimiter=',', 
           header = customHeader, 
           comments = '', 
           fmt='%u')

In [2]:
def preProcessChallenge(dataFrame):
    cP = challenges.groupby(["challenge_id","domain"]).size().reset_index(name="Times")\
        .pivot(index='challenge_id', columns='domain', values='Times').reset_index().drop(["challenge_id"],axis=1)
    cP2 = challenges.groupby(["challenge_id","subdomain"]).size().reset_index(name="Times")\
        .pivot(index='challenge_id', columns='subdomain', values='Times').reset_index().drop(["challenge_id"],axis=1)
    newC = challenges.groupby(["challenge_id"]).size().reset_index(name="Times")
    c = pd.concat([newC,cP,cP2], axis=1)
    challengesDf = c.fillna(0)
    challengesDf = challengesDf.drop(['challenge_id'],axis=1)
    return challengesDf,newC

def preProcessHackers(dataFrame):
    g = submissions.groupby(["hacker_id","contest_id"]).size().reset_index(name="Times")
    gP = submissions.groupby(["hacker_id","contest_id"]).size().reset_index(name="Times")\
        .pivot(index='hacker_id', columns='contest_id', values='Times').reset_index().drop(["hacker_id"],axis=1)
    gP = gP.rename(columns = lambda x : 'contest_id_' + x)
    gP2 = submissions.groupby(["hacker_id","challenge_id"]).size().reset_index(name="Times")\
        .pivot(index='hacker_id', columns='challenge_id', values='Times').reset_index().drop(["hacker_id"],axis=1)
    gP2 = gP2.rename(columns = lambda x : 'challenge_id_' + x)
    gP3 = submissions.groupby(["hacker_id","language"]).size().reset_index(name="Times")\
        .pivot(index='hacker_id', columns='language', values='Times').reset_index().drop(["hacker_id"],axis=1)
    newT = submissions.groupby(["hacker_id"]).size().reset_index(name="Times")
    t = pd.concat([newT,gP,gP2,gP3], axis=1)
    usersDf = t.fillna(0)
    usersDf = usersDf.drop(['hacker_id'],axis=1)
    return usersDf,newT

    

In [4]:
challenges = pd.read_csv('challenges.csv', header=0)  
submissions = pd.read_csv('submissions.csv', header=0)  

challengesDf,challengesIds = preProcessChallenge(challenges)
hackersDf,hackersIds = preProcessHackers(submissions)

In [5]:
from sklearn.metrics import  pairwise

challengeValues = challengesDf.values
challengesDistances = pairwise.linear_kernel(challengeValues)


hackersValues = hackersDf.values
hackersDistances = pairwise.linear_kernel(hackersValues)

In [49]:
def getChallengeSubmitedByHacker(hackerDataFrame,hackerIndex):
    dictOfChallenges =hackerDataFrame.filter(regex=("challenge_id_.*")).iloc[hackerIndex].to_dict()
    dictOfChallengesSubmitedWitheReplace = dict([ (k.replace('challenge_id_',""),v) for k,v in dictOfChallenges.items() if v>0])
    #listOfChallengeOrdered=sorted(dictOfChallengesSubmitedWitheReplace.items(),key=lambda x:x[1],reverse=True)
    return dictOfChallengesSubmitedWitheReplace

def getIntersectAndDisjointChallenges(me,neigbor):
    intersect = []
    disjoint = []
    me_found_id = []
    for k,v in neigbor:
        has = False
        for i,(k2,v2) in enumerate(me):
            if k==k2:
                intersect.append((k,v+v2))
                has =True
                me_found_id.append(k2)
        if not has:
            disjoint.append((k,v))
    disjoint = disjoint+[(k,v) for (k,v) in me if not k in me_found_id]
    disjoint = sorted(disjoint,key=lambda x:x[1],reverse=True)
    intersect = sorted(intersect,key=lambda x:x[1],reverse=True)
    return intersect,disjoint

def getNSimilars(distancesMatrix,meIndex,N=5):
    return np.argsort(-distancesMatrix[meIndex])[0:N]

def getChallengesIdByListIndex(challengesDataFrameWithIndex,challengeListIndex,challengeIdLabel):
    return challengesDataFrameWithIndex.iloc[challengeListIndex][challengeIdLabel].tolist()

def getChallengesIndexByListId(challengesDataFrameWithIdndex,challengeListId,challengeIdLabel):
    return challengesDataFrameWithIdndex.loc[challengesDataFrameWithIdndex[challengeIdLabel].isin(challengeListId)].index.tolist()


In [50]:
resulbyHacker = {}
nHackerSimilars = 1
for i,(hackerId,times) in enumerate(hackersIds.values):
    similars = getNSimilars(hackersDistances,i,nHackerSimilars)
    allChallengesBySimilars = {}
    for indexH in similars:
        dictOfChallengesPartial = getChallengeSubmitedByHacker(hackersDf,indexH)
        for challengeId,value in dictOfChallengesPartial.items():
            if not challengeId in allChallengesBySimilars:
                allChallengesBySimilars[challengeId] = value
            else:
                allChallengesBySimilars[challengeId] = allChallengesBySimilars[challengeId]+ value
    allChallengesBySimilarsOrdered = sorted(allChallengesBySimilars.items(),key=lambda x:x[1],reverse=True)
    listOfChallengesbyUserOrdered = sorted(getChallengeSubmitedByHacker(hackersDf,i).items(),key=lambda x:x[1],reverse=True)
    intersectChallenges,disjointChallenges = getIntersectAndDisjointChallenges(listOfChallengesbyUserOrdered,allChallengesBySimilarsOrdered)
    resulbyChallenge[hackerId] = disjointChallenges


Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

In [43]:
resulbyChallenge = {}
nChallengeSimilars = 5
for i,(hackerId,times) in enumerate(hackersIds.values):
    allChallengesBySimilars = {}
    listOfChallengesbyUser = sorted(getChallengeSubmitedByHacker(hackersDf,i).items(),key=lambda x:x[1],reverse=True)
    lsitOfchallengesIdByUsers = [k for k,v in listOfChallengesbyUser]
    lsitOfchallengesWeightByUsers = [v for k,v in listOfChallengesbyUser]
    listOfIndexChallenges = getChallengesIndexByListId(challengesIds,lsitOfchallengesIdByUsers,'challenge_id')
    for indexChallenge,weight in zip(listOfIndexChallenges,lsitOfchallengesWeightByUsers):
        similars = getNSimilars(challengesDistances,indexChallenge,nChallengeSimilars)
        listOfIdChallengesBylistOfIndex = getChallengesIdByListIndex(challengesIds,similars,'challenge_id')
        for challengeId in listOfIdChallengesBylistOfIndex:
            if not challengeId in allChallengesBySimilars:
                allChallengesBySimilars[challengeId] = value + weight
            else:
                allChallengesBySimilars[challengeId] = allChallengesBySimilars[challengeId]+ value
    allChallengesBySimilarsOrdered = sorted(allChallengesBySimilars.items(),key=lambda x:x[1],reverse=True)
    intersectChallenges,disjointChallenges = getIntersectAndDisjointChallenges(listOfChallengesbyUser,allChallengesBySimilarsOrdered)
    resulbyChallenge[hackerId] = disjointChallenges

In [44]:
nFromUsers = 5
nFromChallenges = 5
finalRecomendations = {}
finalRecomendationsToCsv = []
for i,(hackerId,times) in enumerate(hackersIds.values):
    finalRecomendations[hackerId] = resulbyChallenge[hackerId][0:nFromUsers] + resulbyHacker[hackerId][0:nFromChallenges]
    if not len(finalRecomendations[hackerId]) == 10:
        toTen = 10-len(finalRecomendations[hackerId])
        if len(resulbyChallenge[hackerId][nFromChallenges:])>=toTen:
            finalRecomendations[hackerId] = finalRecomendations[hackerId] + resulbyChallenge[hackerId][nFromChallenges:nFromChallenges+toTen-1]
        elif len(resulbyHacker[hackerId][nFromUsers:])>=toTen:
            finalRecomendations[hackerId] = finalRecomendations[hackerId] + resulbyHacker[hackerId][nFromUsers:nFromUsers+toTen-1]
    toArray = [hackerId] + finalRecomendations[hackerId]
    finalRecomendationsToCsv.append(",".join(map(str,toArray)))

    

{'00004cf8b853ad0d': [('10542da18734d8d9', 161.0),
  ('9d66e1c6d55a467c', 120.0),
  ('f55da2e8f71e3780', 91.0),
  ('85fc279a635f0e92', 80.0),
  ('3acbecb454f4e9f4', 68.0),
  ('f4961978ceb1d212', 68.0),
  ('a30616f47c8e90f8', 65.0),
  ('eee5f9ba016b47aa', 63.0),
  ('32cdf8385434f0dd', 63.0),
  ('433243ab27cc91d0', 52.0),
  ('a1a634218917004f', 47.0),
  ('bf0f60cb6bc59968', 46.0),
  ('d892d2bc59892741', 38.0),
  ('3cea4476faba1071', 35.0),
  ('aed69059f46201b5', 35.0),
  ('d063fb7434821570', 29.0),
  ('803338d7e5cfb022', 29.0),
  ('bd050ea622c35c48', 28.0),
  ('3f7c30ee2dfe74fb', 28.0),
  ('f981d48aa377dd23', 21.0),
  ('bdf8771075b33c2f', 17.0),
  ('2b51eed75e25e273', 10.0),
  ('bd08676326a7951f', 9.0),
  ('3ea1501e8ebd647b', 9.0),
  ('52a1f8e122ccad1a', 7.0),
  ('54d296a59db1bc67', 6.0),
  ('cc437020c5a9164a', 5.0),
  ('8af6cdc91a12dbed', 5.0),
  ('76a423c0ff54a5eb', 5.0),
  ('236dec13a662098a', 5.0),
  ('6c11be05b28edd6c', 4.0),
  ('0f8f6e1babe6b720', 4.0),
  ('638370113f88e004', 4.0),

In [ ]:
saveFileForSubmission(finalRecomendationsToCsv,custonFileName='submission.csv')